In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import Module
import numpy as np
import copy
from torch.nn import functional as F
import numpy as np
from typing import Dict
import pytorch_lightning as pl
from utils import summarize_prune
from pytorch_lightning.loggers import TensorBoardLogger
from provided_code.datasource import DataLoaders
from pytorch_lightning.metrics import functional as FM
from torchvision.datasets import MNIST,CIFAR10
from torchvision import transforms
from torch.utils.data import DataLoader, BatchSampler
from model.cifar10 import  cnn,mlp



### DATA PREPARATION

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

dataset1 = CIFAR10('./data', train=True,
                 transform=transform)
dataset2 = CIFAR10('./data', train=False,
                 transform=transform)
train_loader = DataLoader(dataset1, batch_size=128, shuffle=True, num_workers=4)
test_loader = DataLoader(dataset2, batch_size=32, shuffle=False, num_workers=4)

### MODEL

In [3]:
class Model(pl.LightningModule):
    def __init__(
        self,
        model = None,
        num_classes=10,
        batch_size=32,
        lr=1e-3,
        *args,
        **kwargs
    ):
        super().__init__(*args, **kwargs)

        self.model = model
        self.lr = lr
        self.batch_size = batch_size

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        loss = F.cross_entropy(y_hat, y)
        acc = FM.accuracy(y_hat, y)
        metrics = {
            'loss': loss,
            'acc': acc}
        self.log_dict(metrics)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(
            params=self.model.parameters(),
            lr=self.lr)
        return optimizer


    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        loss = F.cross_entropy(y_hat, y)
        acc = FM.accuracy(y_hat, y)
        metrics = {
            'loss': loss,
            'acc': acc}
        self.log_dict(metrics)
        return metrics



In [4]:
model = Model(model = cnn.CNN())
trainer = pl.Trainer(
    gpus=1,
    fast_dev_run= True)
trainer.fit(model=model,train_dataloader=train_loader,val_dataloaders=test_loader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Running in fast_dev_run mode: will run a full train, val and test loop using 1 batch(es).
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type | Params
-------------------------------
0 | model | CNN  | 62.0 K
-------------------------------
62.0 K    Trainable params
0         Non-trainable params
62.0 K    Total params
0.248     Total estimated model params size (MB)
Epoch 0:  50%|█████     | 1/2 [00:00<00:00,  4.13it/s, loss=2.3, v_num=]
Validating: 0it [00:00, ?it/s]
Epoch 0: 100%|██████████| 2/2 [00:00<00:00,  3.70it/s, loss=2.3, v_num=]


In [15]:
a = Model()
a.optimizers

<bound method LightningModule.optimizers of Model(
  (model): CNN(
    (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (fc1): Linear(in_features=400, out_features=120, bias=True)
    (fc2): Linear(in_features=120, out_features=84, bias=True)
    (fc3): Linear(in_features=84, out_features=10, bias=True)
  )
)>